In [1]:
import os
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import collections
from collections import Counter

In [2]:
def collect_metrics(root_dir):

    depth = 2

    dict_res_roberta = dict()
    dict_res_camem = dict()
    dict_res_xlm = dict()
    
    for subdir, dirs, files in os.walk(root_dir):
        if subdir[len(root_dir):].count(os.sep) < depth:
            for file_ in files:
                if file_ == "xai_metrics.json":
                    with open(subdir + "/" + file_) as f:
                        d = json.load(f)
                        log_odd = d["log_odd"]
                        anti_log_odd = d["anti_log_odd"]
                        comp = d["comp"]
                        suff = d["suff"]
                        
                    model_type = subdir.split(os.sep)[-1].split("_")[0]
                    if model_type == "roberta":
                        dataset_name = " ".join(subdir.split(os.sep)[-1].split("_")[1:])
                        dict_res_roberta[dataset_name] = {"log odd": log_odd, "anti log odd": anti_log_odd, "comp": comp, "suff": suff}
                    elif model_type == "camem":
                        dataset_name = " ".join(subdir.split(os.sep)[-1].split("_")[1:])
                        dict_res_camem[dataset_name] = {"log odd": log_odd, "anti log odd": anti_log_odd, "comp": comp, "suff": suff}
                    elif model_type == "xlm":
                        dataset_name = " ".join(subdir.split(os.sep)[-1].split("_")[1:])
                        dict_res_xlm[dataset_name] = {"log odd": log_odd, "anti log odd": anti_log_odd, "comp": comp, "suff": suff}

    return dict_res_roberta, dict_res_camem, dict_res_xlm

def _average_dicts(dicts):
    avg_dict = collections.defaultdict(lambda: 0)
    
    for i, d in enumerate(dicts):
        for k, v in d.items():
            avg_dict[k] += v
        
    for k, v in avg_dict.items():
        avg_dict[k] /= (i+1)
    
    return avg_dict

def average_over_users(res):
    averaged_en = collections.defaultdict(list)
    averaged_it = collections.defaultdict(list)

    for k, v in res.items():
        if "en" in k and not ("nf" in k):
            averaged_en[" ".join(k.split(" ")[0:2])].append(v)
        elif "it" in k and not ("nf" in k):
            averaged_it[" ".join(k.split(" ")[0:2])].append(v)
    
    result_en = dict()
    result_it = dict()
            
    for k, v in averaged_en.items():
        result_en["avg en " + k] = _average_dicts(v)
        
    for k, v in averaged_it.items():
        result_it["avg it " + k] = _average_dicts(v)
                        
    return result_en, result_it

def process_to_present(res, column_width="1.1cm", plot_transpose=True):
    df_res = pd.DataFrame.from_dict(res).T
    # df_res = df_res.rename(columns=mapping_columns_names)
    if not plot_transpose:
        df_res = df_res.T
    s = df_res.style
    s.format(na_rep='MISS', precision=2)
    print(s.to_latex(column_format='l'+('p{'+f'{column_width}'+'}')*len(df_res.columns)))

In [6]:
dict_res_roberta, dict_res_camem, dict_res_xlm = collect_metrics("../output/sst2_10_40")

## ROBERTA

In [7]:
avg, _ = average_over_users(dict_res_roberta)
roberta_results = {**dict_res_roberta, ** avg}

In [8]:
pd.DataFrame(roberta_results).T

,log odd,anti log odd,comp,suff
np f en6,-0.386056,-0.219160,0.0,0.0
p f en98,-0.378498,-0.509552,0.0,0.0
p f en6,-0.310865,-0.387680,0.0,0.0
p f en83,-0.471155,-0.398874,0.0,0.0
np f en57,-0.366252,-0.667771,0.0,0.0
p f en49,-0.498173,-0.573231,0.0,0.0
p f en57,-0.416160,-0.613106,0.0,0.0
np f en83,-0.316644,-0.708534,0.0,0.0
np f en49,-0.503767,-0.414716,0.0,0.0
np f en98,-0.434014,-0.652388,0.0,0.0


In [9]:
process_to_present(roberta_results)

\begin{tabular}{lp{1.1cm}p{1.1cm}p{1.1cm}p{1.1cm}}
 & log odd & anti log odd & comp & suff \\
np f en6 & -0.39 & -0.22 & 0.00 & 0.00 \\
p f en98 & -0.38 & -0.51 & 0.00 & 0.00 \\
p f en6 & -0.31 & -0.39 & 0.00 & 0.00 \\
p f en83 & -0.47 & -0.40 & 0.00 & 0.00 \\
np f en57 & -0.37 & -0.67 & 0.00 & 0.00 \\
p f en49 & -0.50 & -0.57 & 0.00 & 0.00 \\
p f en57 & -0.42 & -0.61 & 0.00 & 0.00 \\
np f en83 & -0.32 & -0.71 & 0.00 & 0.00 \\
np f en49 & -0.50 & -0.41 & 0.00 & 0.00 \\
np f en98 & -0.43 & -0.65 & 0.00 & 0.00 \\
np nf en & -0.22 & -1.36 & 0.00 & 0.00 \\
p nf en & -0.39 & -0.64 & 0.00 & 0.00 \\
avg en np f & -0.40 & -0.53 & 0.00 & 0.00 \\
avg en p f & -0.41 & -0.50 & 0.00 & 0.00 \\
\end{tabular}



## XLM

In [10]:
avg, _ = average_over_users(dict_res_xlm)
xlm_results = {**dict_res_xlm, ** avg}

In [11]:
pd.DataFrame(xlm_results).T

,log odd,anti log odd,comp,suff
p nf en,-0.278590,-0.474587,0.0,0.0
np f en49,-0.700659,0.001764,0.0,0.0
p f en83,-0.111596,-0.300725,0.0,0.0
p f en49,-0.422125,-0.586324,0.0,0.0
np f en83,-0.716511,-0.029362,0.0,0.0
p f en98,-0.237116,-0.402757,0.0,0.0
np f en98,-0.655847,-0.082400,0.0,0.0
np f en6,-0.643162,-0.055239,0.0,0.0
np nf en,-0.444294,-0.669901,0.0,0.0
p f en57,-0.088318,-0.212464,0.0,0.0


In [9]:
process_to_present(xlm_results)

\begin{tabular}{lp{1.1cm}p{1.1cm}p{1.1cm}p{1.1cm}}
 & log odd & anti log odd & comp & suff \\
p nf en & -0.41 & -0.50 & 0.18 & 0.28 \\
np f en49 & -0.31 & -1.93 & 0.16 & 0.29 \\
p f en83 & -0.14 & -0.34 & 0.03 & 0.11 \\
p f en49 & -0.67 & -0.51 & 0.23 & 0.29 \\
np f en83 & -0.25 & -1.89 & 0.15 & 0.26 \\
p f en98 & -0.48 & -0.40 & 0.17 & 0.21 \\
np f en98 & -0.36 & -1.75 & 0.19 & 0.27 \\
np f en6 & -0.30 & -1.54 & 0.15 & 0.27 \\
np nf en & -0.66 & -1.35 & 0.27 & 0.27 \\
p f en57 & -0.14 & -0.23 & 0.09 & 0.48 \\
p f en6 & -0.22 & -0.24 & 0.14 & 0.33 \\
np f en57 & -0.30 & -1.71 & 0.15 & 0.28 \\
avg en np f & -0.31 & -1.76 & 0.16 & 0.27 \\
avg en p f & -0.33 & -0.34 & 0.13 & 0.28 \\
\end{tabular}

